In [20]:
# !pip install langchain
# !pip install langchain-community
# !pip install langchain-openai
# !pip install beautifulsoup4

In [21]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# FILEPATH: /Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_agents/notebooks/1.3-intro-agents-with-langchain.ipynb
# SET YOUR API KEYS FOR LANGSMITH AND TAVILY TO RUN THIS NOTEBOOK!
# langsmith: https://docs.smith.langchain.com/#:~:text=install%20-U%20langsmith-,Create%20an%20API%20key,click%20Create%20API%20Key.,-Setup%20your%20environment
# tavily: https://app.tavily.com/sign-in
# openai: https://platform.openai.com/docs/quickstart
# import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "your-api-key"
# os.environ["TAVILY_API_KEY"] = "your-api-key"
# os.environ["OPENAI_API_KEY"] = 'your-api-key'

# Building a Research and Report Agent

Steps:

1. Setup the LLM
2. Define the tool or list of tools
3. Set up a prompt template
4. Bind the llm with the tools
6. Use LCEL to connect the agent with prompt and the LLM binded with tools
7. Create the agent loop
8. Wrap everything under `AgentExecutor`
9. Invoke the agent with some query input

## 1. Setup the LLM

In [22]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o")

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e8785592-4c87-46d9-be7d-75949dda0268-0')

## 2. Define the tool or list of tools

In [32]:
import requests
from bs4 import BeautifulSoup
import os

# Tools: search and scraping an internet page
from langchain.tools import tool
from serpapi import GoogleSearch

serpapi_params = {
    "engine": "google",
    "api_key": os.environ["SERPAPI_KEY"]
}


@tool
def search_tool(query: str):
    """Searches the web given a user query and returns the most relevant links"""
    search = GoogleSearch({**serpapi_params, "q": query, "n": 5})
    output = search.get_dict()
    results = output["organic_results"]
    return [r["link"] for r in results]

@tool
def scrape_text(url: str):
    """Scrapes text from an internet page"""
    # source: copied from this script by Harrison Chase https://gist.github.com/hwchase17/69a8cdef9b01760c244324339ab64f0c
    try:
        response = requests.get(url)
        #check
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # Extract all the text from the page
            page_text = soup.get_text(separator=" ", strip=True)
            
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code: {response.status_code}"
    
    except Exception as e:
        # In case of error print the error message
        print(e)
        return f"Failed to retrieve the webpage: {e}"

query = "Trends in using AI for education."
links = search_tool.invoke(query)
print(links)

url = links[0]

print(scrape_text.invoke(url))

['https://www.the74million.org/article/7-artificial-intelligence-trends-that-could-reshape-education-in-2024/', 'https://www.hurix.com/top-trends-reshaping-the-world-of-education-in/', 'https://www.forbes.com/sites/forbescommunicationscouncil/2024/01/23/how-ai-and-data-will-change-education-in-2024/', 'https://caktus.ai/blog/ai-in-education', 'https://www.weforum.org/agenda/2024/04/future-learning-ai-revolutionizing-education-4-0/', 'https://www2.ed.gov/documents/ai-report/ai-report.pdf', 'https://www.panopto.com/blog/five-ai-trends-that-are-good-news-for-higher-education/', 'https://www.sciencedirect.com/science/article/pii/S1877050918315382', 'https://elearningindustry.com/how-ai-is-impacting-the-way-we-learn-key-trends-and-insights']
7 Artificial Intelligence Trends That Could Reshape Education in 2024 – The 74 The 74 America's Education News Source About The 74 Donate 7 Artificial Intelligence Trends That Could Reshape Education in 2024 News Opinion Home Video Analysis Search Donat

In [34]:
@tool
def write_final_report(markdown_report: str):
    """Writes the final report to a file"""
    with open("final_report.md", "w") as file:
        file.write(markdown_report)
    
    return markdown_report

In [38]:
tools = [search_tool, scrape_text]

3. Set up a prompt template


In [39]:
from langchain.prompts import ChatPromptTemplate

WRITER_SYSTEM_PROMPT = "You are an AI critical thinker research assistant. \
                        Your sole purpose is to write well written, critically \
                        acclaimed, objective \
                        and structured reports on given text."
# Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """Information:
Write a detailed research report on the following question:\n
"{input}". -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("placeholder"), "{chat_history}",
        ("human", RESEARCH_REPORT_TEMPLATE),
        ("placeholder"), "{agent_scratchpad}",
    ]
)

4. Bind the llm with the tools


In [40]:
llm_with_tools = llm.bind_tools(tools)

In [26]:
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.agents.output_parsers import ToolsAgentOutputParser

5. Use LCEL to define and connect agent, prompt and the LLM binded with tools


In [51]:
from langchain_core.runnables import RunnablePassthrough

agent = (
RunnablePassthrough.assign(
                    agent_scratchpad=lambda x: format_to_tool_messages(x['intermediate_steps'])
                    )
    | prompt
    | llm_with_tools
    | ToolsAgentOutputParser()
)

6. Create the agent loop

7. Wrap everything under `AgentExecutor`s

In [52]:
from langchain.agents import AgentExecutor

In [53]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

8. Invoke the agent with some query input

In [56]:
from IPython.display import Markdown

output_report = agent_executor.invoke({"input": "Write a quick report on the current trends in AI. Focus on LLMs for education.", "chat_history": []})

Markdown(output_report["output"])



> Entering new AgentExecutor chain...

Invoking: `search_tool` with `{'query': 'current trends in AI'}`


['https://www.ibm.com/blog/artificial-intelligence-trends/', 'https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends', 'https://www.eweek.com/artificial-intelligence/ai-trends/', 'https://www.simplilearn.com/artificial-intelligence-ai-and-machine-learning-trends-article', 'https://www.forbes.com/sites/bernardmarr/2023/09/18/the-10-most-important-ai-trends-for-2024-everyone-must-be-ready-for-now/', 'https://www.coursera.org/articles/ai-trends', 'https://www.synthesia.io/post/ai-trends', 'https://sloanreview.mit.edu/article/five-key-trends-in-ai-and-data-science-for-2024/', 'https://appinventiv.com/blog/ai-trends/']
Invoking: `scrape_text` with `{'url': 'https://www.ibm.com/blog/artificial-intelligence-trends/'}`


The most important AI trends in 2024 - IBM Blog The most important AI trends in 2024 Artificial intelligence Business transformation Febr

# Current Trends in AI: A Focus on Large Language Models (LLMs) for Education

## Introduction

Artificial Intelligence (AI) has been a transformative force across various sectors, and its influence continues to grow. This report will focus on the current trends in AI, with a particular emphasis on Large Language Models (LLMs) for education. The information presented is based on a comprehensive review of various sources, including articles from IBM, the University of Illinois, and other reputable sources.

## Current Trends in AI

AI has been evolving rapidly, with generative AI gaining significant attention in recent years. Generative AI refers to AI models that can generate new content, such as text, images, or music, based on the data they have been trained on. Large Language Models (LLMs) are a type of generative AI that can generate human-like text, making them particularly useful in various applications, including education.

According to IBM, 2024 is a pivotal year for AI, with generative AI becoming more integrated into our everyday lives. The evolution of generative AI has mirrored that of computers, albeit on a dramatically accelerated timeline. Generative AI has already reached its “h